# Transportation Infrastructure

In this notebook, we will perform a **damage and risk assessment** for transportation infrastructure, specifically focusing on roads. The assessment is based on combining hazard data (e.g., flood depths) with vulnerability curves to estimate the potential damage to infrastructure.

We will follow the steps outlined below to conduct the assessment:

1. **Loading the necessary packages:**  
   We will import the Python libraries required for data handling, analysis, and visualization.

2. **Loading the data:**  
   The infrastructure data (e.g., roads) and hazard data (e.g., flood depths) will be loaded into the notebook.

3. **Preparing the data:**  
   The infrastructure and hazard data will be processed and data gaps can be filled, if required.

4. **Performing the damage assessment:**  
   We will apply vulnerability curves to estimate the potential damage based on the intensity of the hazard.

5. **Visualizing the results:**  
   Finally, we will visualize the estimated damage using graphs and maps to illustrate the impact on transportation infrastructure.

## 1. Loading the Necessary Packages
In this step, we will import all the required Python libraries for data manipulation, spatial analysis, and visualization.

In [1]:
import os,sys
import rasterio
import shapely
import xarray as xr

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

from tqdm import tqdm

from pathlib import Path
import matplotlib.pyplot as plt
import rioxarray
import contextily as cx

from osm_flex import download
from exactextract import exact_extract

### Specify the country of interest

Before we continue, we should specify the country for which we want to assess the damage. We use the ISO3 code for the country to download the OpenStreetMap data.

In [2]:
country_full_name = 'Jamaica'
country_iso3 = 'JAM'

## 2. Loading the Data
In this step, we will load four key datasets:

1. **Infrastructure data:**  
   This dataset contains information on the location and type of transportation infrastructure (e.g., roads). Each asset may have attributes such as type, length, and replacement cost.

2. **Hazard data:**  
   This dataset includes information on the hazard affecting the infrastructure (e.g., flood depth at various locations).

3. **Vulnerability curves:**  
   Vulnerability curves define how the infrastructure's damage increases with the intensity of the hazard. For example, flood depth-damage curves specify how much damage occurs for different flood depths.

4. **Maximum damage values:**  
   This dataset contains the maximum possible damage (in monetary terms) that can be incurred by individual infrastructure assets. This is crucial for calculating total damage based on the vulnerability curves.

### Infrastructure Data

We will perform this example analysis for Jamaica. To start the analysis, we first download the OpenStreetMap data from GeoFabrik. 

In [3]:
infrastructure_path = download.get_country_geofabrik(country_iso3)

Now we load the data and read only the road data.

In [4]:
%%time
features = gpd.read_file(infrastructure_path, layer="lines")
features = features[features["highway"].values != None]
features = features.to_crs(3857)

CPU times: total: 3.97 s
Wall time: 5.31 s


In [5]:
features = features[features['highway'].isin(['motorway', 'motorway_link', 'trunk', 'trunk_link',
                            'primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
                            'tertiary_link', 'residential', 'road', 'unclassified','track'])].reset_index(drop=True)

### Hazard Data
For this example, we make use of the flood data provided by [CDRI](https://giri.unepgrid.ch/map?list=explore).

We use a 1/100 flood map to showcase the approach.

In [ ]:
hazard_map = xr.open_dataset("https://hazards-data.unepgrid.ch/global_pc_h100glob.tif", engine="rasterio")

### Maximum damages
One of the most difficult parts of the assessment is to define the maximum reconstruction cost of particular assets. Here we provide a baseline set of values, but these should be updated through national consultations.

In [ ]:
maxdam_dict = {'unclassified':300, 
        'primary':2000, 
        'secondary':1300, 
        'tertiary':700, 
        'residential':500,
        'trunk':2000, 
        'trunk_link':2000, 
        'motorway':2000, 
        'motorway_link':2000, 
        'primary_link':2000, 
        'secondary_link':1300,
        'tertiary_link':700,
        'road':700,
        'track':300, }

### Vulnerability data
Similarly to the maximum damages, specifying the vulnerability curves is complex. We generally have limited information about the quality of the assets, its level of deteriation and other asset-level characteristics.

In [ ]:
vulnerability_path = "https://zenodo.org/records/10203846/files/Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx?download=1"
vul_df = pd.read_excel(vulnerability_path,sheet_name='F_Vuln_Depth')

road_curves = vul_df[['ID number','F7.1']]
road_curves = road_curves.iloc[4:125,:]
road_curves.set_index('ID number',inplace=True)
road_curves.index = road_curves.index.rename('Depth')  

### Ancilliary data for processing

In [ ]:
world = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_admin_0_countries.shp")
world_plot = world.to_crs(3857)

## 3. Preparing the Data

Change the column name of the object data, to ensure we can use the same functions across different infrastructure assets

In [ ]:
features = features.rename(columns = {'highway' : 'object_type'})

Clip the hazard data to the country of interest.

In [ ]:
country_bounds = world.loc[world.ADM0_ISO == country_iso3].bounds
country_geom = world.loc[world.ADM0_ISO == country_iso3].geometry

In [ ]:
hazard_country = hazard_map.rio.clip_box(minx=country_bounds.minx.values[0],
                     miny=country_bounds.miny.values[0],
                     maxx=country_bounds.maxx.values[0],
                     maxy=country_bounds.maxy.values[0]
                    )

And reproject it to **EPSG:3857** so we ensure that we can overlay this with the features we will analyse, and have all the overlay in meters.

In [ ]:
%%time
hazard_country = hazard_country.rio.reproject("EPSG:3857")

Let's check the extent of the flood and the units in the data

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 3))

hazard_country.band_data.plot(ax=ax,cmap='Blues')
world_plot.loc[world.SOV_A3 == country_iso3].plot(ax=ax,facecolor="none",edgecolor='black')
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)

## 4. Performing the Damage Assessment
We will apply vulnerability curves to the exposed infrastructure to estimate the damage based on the intensity of the hazard. The vulnerability curve translates hazard intensity (e.g., flood depth) into damage potential.

In [ ]:
%%time
# get each unique severity of the hazard per object
values_and_coverage_per_object = exact_extract(
    hazard_country, features, ["coverage", "values"], output="pandas"
)

In [ ]:
features[features['coverage'].map(len) > 0]

In [ ]:
features = features.merge(values_and_coverage_per_object,left_index=True,right_index=True)

In [ ]:
hazard_intensity = np.float16(road_curves.index.values)*100
fragility_values = np.float16(road_curves['F7.1'].values)

In [ ]:
def _get_damage_per_element(asset,hazard_intensity,fragility_values,maxdam_dict):
    """
    Calculate damage for a given asset based on hazard information.
    Arguments:
        *asset*: Tuple containing information about the asset. It includes:
            - Index or identifier of the asset (asset[0]).
            - Asset-specific information, including hazard points (asset[1]['hazard_point']).
        *maxdam_dict*: Maximum damage value.
    Returns:
        *tuple*: A tuple containing the asset index or identifier and the calculated damage.
    """
    
    return np.sum(np.interp(np.float16(asset['values']),
       hazard_intensity,fragility_values)*asset['coverage']*maxdam_dict[asset['object_type']])
   

In [ ]:
tqdm.pandas()
features['damage'] = features.progress_apply(lambda asset : 
                                                     _get_damage_per_element(asset,
                                                    hazard_intensity, fragility_values,maxdam_dict),axis=1) 

## 5. Visualizing the Results
The results of the damage assessment can be visualized using charts and maps. This will provide a clear representation of which infrastructure is most affected by the hazard and the expected damage levels.

In [ ]:
damage_results = features.loc[features.damage > 0]

Find the locations of the twenty most damaged roads.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

damage_results.sort_values('damage',ascending=False).head(20).plot(ax=ax,color='Red')
features.plot(ax=ax,facecolor="none",edgecolor='grey',alpha=0.5,zorder=2,lw=0.1)
world_plot.loc[world.SOV_A3 == country_iso3].plot(ax=ax,facecolor="none",edgecolor='black')
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)
ax.set_axis_off()

And create a distribution of the damages.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7, 3))

sns.histplot(data=damage_results,x='damage',ax=ax)

Plot all damages

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

damage_results.plot(ax=ax,column='damage',cmap='Reds',legend=True,
                   legend_kwds={'shrink': 0.5,'label':'Absolute Damage'},zorder=5)
features.plot(ax=ax,facecolor="none",edgecolor='grey',alpha=0.5,zorder=2,lw=0.1)
world_plot.loc[world.SOV_A3 == country_iso3].plot(ax=ax,facecolor="none",edgecolor='black')
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron,alpha=0.5)
ax.set_axis_off()